In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
data = []
DataDir = '/kaggle/input/cat-and-dog/training_set/training_set'
types = ['cats','dogs']
img_size = 50
for type in types:
    path = os.path.join(DataDir,type)
    type_num = types.index(type)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array,(img_size,img_size))
            data.append([new_array,type_num])
        except:
            print("bad file")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = np.array(data)

In [31]:
data.shape
t = data.shape[0]
training_data,testing_data = data[:int(3*t/4)],data[int(t/4):]

In [35]:
x_train = []
y_train = []
for i in range(training_data.shape[0]):
    x_train.append(training_data[i][0])
    y_train.append(training_data[i][1])

x_train = np.array(x_train)
y_train = np.array(y_train).reshape(-1,1).T
print(f'x_train = {x_train.shape}\ny_train = {y_train.shape}')

In [37]:
plt.imshow(x_train[4])

In [44]:
x_train = x_train.reshape(-1,img_size*img_size*3).T
x_train.shape


In [45]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def ReLU(Z):
    return np.maximum(Z,0)

In [46]:
layer_dims = [x_train.shape[0],20,5,1]
def initialize_parameters_deep(layer_dims):
    params = []
    for i in range(1,len(layer_dims)):
        W =  np.random.randn(layer_dims[i],layer_dims[i-1])*0.01
        b = np.zeros((layer_dims[i],1))*0.01

        dic = {
            "W":W,
            "b":b
        }
        params.append(dic)
    #eg: params = [W1,b1,W2,b2....]
    # print(f'params = {params}')
    return params

def linear_activation_forward(A_prev,W,b, activation):
    
    print(f'shape of W,A_prev = {W.shape}, {A_prev.shape}')

    Z = np.dot(W,A_prev) + b

    print(f'shape of Z = {Z.shape}')
    
    cache = (A_prev,W,b)
    if activation == "sigmoid":
        A = sigmoid(Z)

    elif activation == "relu":
        A = ReLU(Z)

    return A, cache 

def L_model_forward(X,params):
    #size of layer
    L = len(params)-1
    caches = []
    A = X

    print(f'length of params = {L}')
    for l in range(0,L):
        A_prev = A
        W,b = params[l]["W"],params[l]["b"]
        A,cache = linear_activation_forward(A_prev,W,b,"relu")
        caches.append(cache)

    AL, cache = linear_activation_forward(A,params[L]["W"],params[L]["b"],"sigmoid")
    caches.append(cache)

    return AL, caches


def compute_cost(AL,Y):
    m = Y.shape[1]
    
    print(f'shape of AL,Y = {AL.shape}, {Y.shape}')
    cost = -1/m * np.sum(np.multiply(Y,np.log(AL)) + np.multiply(1-Y,np.log(1-AL)))
    cost = np.squeeze(cost)

    return cost

In [50]:
#backward prop
def relu_backward(dZ_prev, cache):
    A,b,b = cache
    A[A>=0] = 1
    A[A<0] = 0

    dZ = np.multiply(dZ_prev,A)
    return dZ

def sigmoid_backwards(dZ_prev, cache):
    dZ = np.multiply(dZ_prev,(1-dZ_prev))
    return dZ

def L_activation_backward(dA, cache, activation):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    if activation == "relu":
        dZ = relu_backward(dA, cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backwards(dA,cache)

    dW = 1/m* np.dot(dZ,A_prev.T)
    db = 1/m* np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)

    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    # Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = L_activation_backward(dAL,current_cache,"sigmoid")

    grads["dA"+str(L-1)] = dA_prev_temp
    grads["dW"+ str(L)] = dW_temp
    grads["db"+ str(L)] = db_temp

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = L_activation_backward(dA_prev_temp,current_cache,"relu")

        grads["dA"+str(l)] = dA_prev_temp
        grads["dW"+str(l+1)] =  dW_temp
        grads["db"+str(l+1)] = db_temp
    
    return grads

def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters)//2
    
    for l in range(L):
        parameters[l]["W"] = parameters[l]["W"] - learning_rate*grads["dW"+str(l)]
        parameters[l]["b"] = parameters[l]["b"] - learning_rate*grads["db"+str(l)]

    return parameters
        

In [51]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):
        
        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL,Y)
        print(cost)

        grads = L_model_backward(AL,Y,caches)
        
        # UPDATE parameters
        parameters = update_parameters(parameters, grads, learning_rate)
        
                
        # Print the cost every 100 iterations
        # if print_cost and i % 1 == 0 or i == num_iterations - 1:
        print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [ ]:
parameters,costs = L_layer_model(x_train,y_train,layer_dims,num_iterations=100,print_cost=True)